# Load complex_system.xml and make a `Simulation` obj

We will ignore equilibration, which is something you probably don't want to do in production.

In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

import numpy as np

from sys import stdout
import sys
from tqdm import tqdm



In [2]:
#load complex
complex_pdb = PDBFile('./openmm_simulation/complex_coords.pdb')
complex_system = XmlSerializer.deserialize(open('./openmm_simulation/complex_system_.xml').read())



In [3]:
##add a barostat. Not only will this initially fix any vacuum between periodic cells,
##it will be maintained throughout to keep pressure reasonable. 
barostat = MonteCarloBarostat(1*bar, 310*kelvin)
complex_system.addForce(barostat)

5

In [4]:
#OpenMM simulation machinery:
integrator = LangevinIntegrator(310*kelvin, 10/picosecond, 0.002*picoseconds)
platform = Platform.getPlatformByName('OpenCL')
prop = {'OpenCLPrecision':'single', 'OpenCLDeviceIndex':'0'}
simulation = Simulation(complex_pdb.topology, complex_system, integrator, platform, prop)


In [5]:
#minimize starting coords:
simulation.context.setPositions(complex_pdb.positions)
simulation.minimizeEnergy(maxIterations=150)

# Add a DCDReporter, and simulate
takes about three hours for 

In [ ]:
simulation.reporters.append(DCDReporter('./traj_production.dcd', 10000)) #20ps per sample
simulation.reporters.append(StateDataReporter(stdout, 10000, 
                                              step=True, 
                                              potentialEnergy=True, 
                                              temperature=True, speed=True))


##Extra 1 ns for good measure.
step_per_ns = 500000
num_ns = 5
simulation.step(step_per_ns * num_ns)


#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
10000,-244065.23634090647,311.14452014330993,0
20000,-245238.6391195343,311.2367601593027,320
30000,-244572.41162934923,311.21327211910284,318
40000,-243526.35017299443,309.7162505670558,317
50000,-245036.13331117085,310.2357222430113,318
60000,-244498.9488313219,307.13913711712553,318
70000,-245268.86636492494,307.829644307307,319
80000,-244202.03798961733,309.34086661550634,319
90000,-244678.8395471482,311.92254423500805,319
100000,-245656.95003902237,307.689400812341,320
110000,-244749.3330604455,311.8199753375892,319
120000,-245446.59819297493,312.7549370825635,318
130000,-245292.8781363119,308.7991431064189,318
140000,-245350.94295240124,313.9389951268412,317
150000,-244884.48218487552,307.3702998864197,317
160000,-244187.17482831865,312.760245831059,317
170000,-245248.16043884563,308.2519124197746,317
180000,-245565.93572591315,311.77475179223717,316
